This code is to show the use of the DNN regressor for reggression problems

In [17]:
import tensorflow as tf
import tensorflow.feature_column as fc
import csv
import sys
import os
import pandas as pd
import numpy as np

In [18]:
LAB_PATH=os.path.abspath(os.curdir)
DATASET_PATH=os.path.join(LAB_PATH,"datasets")
file_path=os.path.join(DATASET_PATH,"ds1.csv")
BATCH_SIZE=32
STEPS=1000
NUM_EPOCHS=40
data = pd.read_csv(file_path)
# print("before",data)
# x_norm_factor=data["Input"].max()-data["Input"].min()
# y_norm_factor=data["Output"].max()-data["Output"].min()
# print(x_norm_factor,y_norm_factor)
# data=data/(data.max()-data.min())
# print("after",data)

In [19]:
# get the two dataset columns as numpy arrays

def get_input():
    # get the two dataset columns as numpy arrays
    # normalize all the values
    #data[['Input','Output']] /= data[['Input','Output']].max()
    global data
    #data=(data-data.mean())/data.std()
    #print("after",data)
    train_x = data['Input'].values
    train_y = data['Output'].values

    dataset = tf.data.Dataset.from_tensor_slices(({'Input':train_x}, train_y))

    # apply transformations to the dataset
    dataset = dataset.batch(BATCH_SIZE)  # get the data in batches of BATCH_SIZE
    dataset = dataset.repeat(NUM_EPOCHS) # need enough rows for training
    return dataset


In [20]:
# Estimator using the default optimizer.
def train_dnn_reg():
    estimator = tf.estimator.DNNRegressor(feature_columns=[fc.numeric_column('Input')],hidden_units = [64,32,16])
    estimator.train(input_fn = get_input)
    metrics=estimator.evaluate(input_fn=get_input)
    return estimator
    

In [21]:
est=train_dnn_reg()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_master': '', '_session_config': None, '_num_ps_replicas': 0, '_save_checkpoints_steps': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000025FFC3A7630>, '_global_id_in_cluster': 0, '_train_distribute': None, '_num_worker_replicas': 1, '_model_dir': 'C:\\Users\\Admin\\AppData\\Local\\Temp\\tmp5z2jujdb', '_evaluation_master': '', '_save_checkpoints_secs': 600, '_device_fn': None, '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_save_summary_steps': 100, '_is_chief': True, '_tf_random_seed': None, '_task_id': 0, '_service': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 int

INFO:tensorflow:global_step/sec: 1052.99
INFO:tensorflow:loss = 0.004443771, step = 7001 (0.096 sec)
INFO:tensorflow:global_step/sec: 970.452
INFO:tensorflow:loss = 0.00046245518, step = 7101 (0.104 sec)
INFO:tensorflow:global_step/sec: 929.097
INFO:tensorflow:loss = 0.00033744378, step = 7201 (0.106 sec)
INFO:tensorflow:global_step/sec: 1002
INFO:tensorflow:loss = 0.00042695977, step = 7301 (0.103 sec)
INFO:tensorflow:global_step/sec: 949.965
INFO:tensorflow:loss = 0.0009402836, step = 7401 (0.106 sec)
INFO:tensorflow:global_step/sec: 938.275
INFO:tensorflow:loss = 0.0014605138, step = 7501 (0.104 sec)
INFO:tensorflow:global_step/sec: 970.297
INFO:tensorflow:loss = 0.0003589854, step = 7601 (0.100 sec)
INFO:tensorflow:global_step/sec: 952.649
INFO:tensorflow:loss = 0.0007700585, step = 7701 (0.109 sec)
INFO:tensorflow:global_step/sec: 949.326
INFO:tensorflow:loss = 0.00063783705, step = 7801 (0.102 sec)
INFO:tensorflow:global_step/sec: 948.14
INFO:tensorflow:loss = 0.0006190466, step 

In [22]:
def denormalise(pred,bias,ele):
    pred*=y_norm_factor
    bias*=y_norm_factor
    ele*=x_norm_factor
    return pred,bias,ele



def get_equation(est):
    global data
    train_x = data['Input'].values[:10]  # as a numpy array
    train_y = data['Output'].values[:10]
    ele=train_x[1]#its normalised as here!
    print(train_x[1],train_y[1])
    #est = tf.estimator.LinearRegressor(feature_columns=[fc.numeric_column('Input')])
    #bias=list(estimator.predict(input_fn = get_input))
    zero_inp=tf.estimator.inputs.numpy_input_fn(x = {'Input':np.array([0.0],dtype = np.float64)}, shuffle = False)
    rand_inp=tf.estimator.inputs.numpy_input_fn(x = {'Input':np.array([ele],dtype = np.float64)}, shuffle = False)
    bias=list(est.predict(zero_inp))[0]['predictions'][0]
    pred=list(est.predict(rand_inp))[0]['predictions'][0]
    pred,bias,ele=denormalise(pred,bias,ele)
    slope=(pred-bias)/ele
    print("bias is ",bias,"pred is",pred,"slope is",slope)
    print("The equation is-->","y = ",slope,"x + ",bias)

    
#F = 1.8*C + 32 
get_equation(est)

1.879264813 0.6828327259999999
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Admin\AppData\Local\Temp\tmp5z2jujdb\model.ckpt-12520
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Admin\AppData\Local\Temp\tmp5z2jujdb\model.ckpt-12520
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
bias is  0.0036753905858107573 pred is 0.9880274444127122 slope is 0.08337597518540833
The equation is--> y =  0.08337597518540833 x +  0.0036753905858107573
